## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 32, 32, 16)   64          conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 32, 32, 16)   0           batch_normalization_29[0][0]     
__________________________________________________________________________________________________
conv2d_33 

In [5]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [10]:
def generate_arrays_from_file(feature=x_train, label=y_train, batch_train_size = 1024):
    while True:
        for index in range(0, len(feature), batch_size):
            yield feature[index:index+batch_size], label[index:index+batch_size]
        for index in range(0, len(feature), batch_size):
            yield next(data_generator.flow(x_train, y_train, batch_size, shuffle=True))

In [12]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練整個資料集共 30個循環
from math import ceil

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit_generator(generate_arrays_from_file(x_train,y_train), steps_per_epoch=ceil(x_train.shape[0] / batch_size * 2), 
                              epochs=epochs, 
                              verbose=1, 
                              validation_data=(x_test, y_test), 
                              class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
782/782 [==============================] - 4380s 6s/step - loss: 1.7368 - acc: 0.5456 - val_loss: 2.1435 - val_acc: 0.4847
Epoch 2/10
782/782 [==============================] - 4291s 5s/step - loss: 1.2788 - acc: 0.6752 - val_loss: 2.9794 - val_acc: 0.4102
Epoch 3/10
782/782 [==============================] - 4257s 5s/step - loss: 1.0680 - acc: 0.7376 - val_loss: 1.5204 - val_acc: 0.6357
Epoch 4/10
782/782 [==============================] - 4258s 5s/step - loss: 0.9453 - acc: 0.7767 - val_loss: 1.6610 - val_acc: 0.5990
Epoch 5/10
782/782 [==============================] - 4263s 5s/step - loss: 0.8565 - acc: 0.8047 - val_loss: 1.0571 - val_acc: 0.7483
Epoch 6/10
782/782 [==============================] - 4263s 5s/step - loss: 0.7948 - acc: 0.8227 - val_loss: 1.4472 - val_acc: 0.6810
Epoch 7/10
782/782 [==============================] - 4261s 5s/step - loss: 0.7550 - acc: 0.8357 - val_loss: 1.7423 - val_acc: 0.6139
Epoch 8/10
782/782 [==============================] - 4549s 6s